In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse
from scipy.stats import multivariate_normal
from scipy.stats import norm

In [55]:
# Generamos el dataset, blobs con distinta varianza
n_samples = 1500
X, y = datasets.make_blobs(n_samples=n_samples, cluster_std=[.8, 2, 0.4], random_state=1)
X = np.array(X[:, ::-1][:,0])
X = X.reshape(-1,1)
X.shape

(1500, 1)

## Utilizamos la función para GMM provista por Sklearn

In [57]:
gm = GaussianMixture(n_components=2, covariance_type='full', random_state=7)
gm.fit(X,y)

GaussianMixture(n_components=2, random_state=7)

In [71]:
gm.means_
gm.covariances_

print("Medias y covarianzas utilizando SKLEARN\n")

print("Medias: ")
print(gm.means_, "\n")

print("Covarianzas: ")
print(gm.covariances_, "\n")

Medias y covarianzas utilizando SKLEARN

Medias: 
[[ 4.42416944]
 [-6.03602157]] 

Covarianzas: 
[[[0.63660982]]

 [[6.62270836]]] 



## Generamos un algoritmo con Numpy para calcular GMM

In [73]:
# Generamos un algoritmo con Numpy
class BaseModel(object):

    def __init__(self):
        self.model = None

    def fit(self, X, Y):
        return NotImplemented

    def predict(self, X):
        return NotImplemented
class GMM(BaseModel):
    
    def fit(self, X, y):

        # Dimensions
        n = X.shape[0]
        k = 2

        # Parameters initialization
        p = np.random.uniform(0, 1, (k, 1))
        p = p / np.sum(p, axis=0)
        means = np.random.uniform(np.min(X), np.max(X), (k, 1))
        covariance = np.sum((np.hstack((X, X))-means.T)**2, axis=0)/(X.shape[0]-1)
        covariance = covariance.reshape(-1, 1)
        Nij = np.zeros((n, k))
        Eij = np.zeros((n, k))
        Eij_ant = np.zeros((n, k))

        i = 0
        delta = False
        tol = 1E-5
        for j in range(k):
            Nij[:, j] = multivariate_normal.pdf(X, means[j], covariance[j])

        # Execution Loop
        while not (delta or i > 500):
            Eij_ant[:, :] = Eij
            for j in range(k):
                Eij[:, j] = (p[j] * Nij[:, j]) / (Nij @ p)[:, 0]
                means[j] = (Eij[:, j].dot(X)) / np.sum(Eij[:, j], axis=0)
                covariance[j] = Eij[:, j].dot((X - means[j]) * (X - means[j])) / np.sum(Eij[:, j])
                p[j] = np.mean(Eij[:, j])
                Nij[:, j] = multivariate_normal.pdf(X, means[j], covariance[j])
            delta = np.allclose(Eij_ant, Eij, rtol=tol)
            i = i + 1
        idx = np.argsort(means[:, 0], axis=0)
        self.model = {'mu': means[idx, :], 'cov': covariance[idx, :], 'p': p[idx, :]}

    def predict(self, X):
        k = self.model['mu'].shape[0]
        N = np.zeros((X.shape[0], k))
        E = np.zeros((X.shape[0], k))

        for i in range(k):
            N[:, i] = multivariate_normal.pdf(X, self.model['mu'][i, 0], self.model['cov'][i, 0])
        for i in range(k):
            E[:, i] = (self.model['p'][i, 0] * N[:, i]) / (N @ self.model['p'])[:, 0]
        idx = np.argmax(E, axis=1)
        return idx

In [76]:
# Aplicamos el algoritmo a nuestro dataset generado
EM = GMM()
EM.fit(X, y)
predictions = EM.predict(X)

In [75]:
print("Medias y covarianzas utilizando el algoritmo generado con Numpy")

print("Medias: ")
print(EM.model['mu'], "\n")

print("Covarianzas: ")
print(EM.model['cov'], "\n")



Medias y covarianzas utilizando el algoritmo generado con Numpy
Medias: 
[[-6.0358446 ]
 [ 4.42425893]] 

Covarianzas: 
[[6.62405345]
 [0.63642695]] 



## Conclusión: Vemos que los valores de medias y covarianzas utilizando ambos métodos son casi idénticos!